# Face Detection Pipeline

I am building a Face Detection Pipeline using the Open CV frontal face Haar cascade detection model (as we did in week 4). 

I will improve the output in two steps:
1. Remove multiple boxes for the same face using Non-Maximum Suppression (NMS).
2. Verify that each box actually contains a face with our trained HIOG feature classifier from week 5. 




## Architecture

I will use a single class "FaceDetector" to run all steps of the pipeline. 

I think the task is too defined to use a configurable Pipeline class, and I think the individual steps are too straightforward to justify individual classes for Detection, Cleanup and Verification. 